# Google colab requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Thesis/test')
os.getcwd()

In [ ]:
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric

# Hyperparameters

In [1]:
hidden_channels = 256
beta = 0.001
dropout = 0.5
lr = 0.01
num_epochs = 500
do_tsne = False

# x_RBM
x_RBM_num_hid = 4096
x_RBM_num_epochs = 1000
x_RBM_lr = 0.001
x_RBM_num_CD = 1
x_RBM_batch_size= 64

# z0_RBM
z0_RBM_num_hid = 4096
z0_RBM_num_epochs = 1000
z0_RBM_lr = 0.001
z0_RBM_num_CD = 1
z0_RBM_batch_size= 64

# z1_RBM
z1_RBM_num_hid = 4096
z1_RBM_num_epochs = 1000
z1_RBM_lr = 0.001
z1_RBM_num_CD = 1
z1_RBM_batch_size= 64

# z2_RBM
z2_RBM_num_hid = 4096
z2_RBM_num_epochs = 1000
z2_RBM_lr = 0.001
z2_RBM_num_CD = 1
z2_RBM_batch_size= 64

Libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import plotly.express as px
from bokeh.palettes import Category20_20, Category20b_20, Accent8

from tqdm import tqdm
import sys
sys.path.insert(0, "../..") # denoiseRBM
sys.path.insert(1, "..") # examples

import os
import os.path as osp

from denoiseRBM.distort_X import distort_x
from denoiseRBM.distort_A import distort_a
from denoiseRBM.RBM import B_RBM
from denoiseRBM.utils import Z, plot_tsne
from denoiseRBM.denoiseRBM import denoiseRBM
from ogbn_arxiv import ogbn_arxiv, test
from DNN import DNN, train

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Z_ = Z(d=hidden_channels, beta=beta)

results = {
    # Corrupted data
    "X_c": {
        "phi": {},
        "psi_0": {},
        "psi_1": {},
        "psi_2": {}
    },
    # Blanked out data
    "X_z": {
        "phi": {},
        "psi_0": {},
        "psi_1": {},
        "psi_2": {}
    }
}

Paths to store results

In [4]:
chkpt_path = "MLP"
if not osp.exists(chkpt_path):
    os.mkdir(chkpt_path)

Initialize ogbn-arxiv and MLP model

In [ ]:
x, y_true, edge_index, idx, nodes, x_distorted, A_distorted, evaluator = ogbn_arxiv("data")
MLP_model = DNN(NN=nn.Linear, in_channels=x.size(-1), hidden_channels=hidden_channels, 
                out_channels=len(np.unique(y_true.numpy())), device=device, mathcal_Z=None,
                dropout=0.5)

Distorting edge index:   0%|                                                                  | 0/10.0 [00:00<?, ?it/s]
0.0 distortion:   1%|▍                                                       | 10008/1166243 [00:06<17:36, 1094.12it/s]


0.0 distortion:   2%|▉                                                       | 20409/1166243 [00:14<08:58, 2128.53it/s]


0.0 distortion:   3%|█▌                                                      | 32526/1166243 [00:21<08:14, 2293.35it/s]


0.0 distortion:   4%|██▎                                                     | 48624/1166243 [00:28<09:04, 2052.53it/s]


0.0 distortion:   5%|██▉                                                     | 62105/1166243 [00:35<12:35, 1461.12it/s]


...

0.0 distortion:  73%|████████████████████████████████████████▎              | 854932/1166243 [06:12<02:32, 2035.21it/s]


0.0 distortion:  75%|████████████████████████████████████████▉              | 869039/1166243 [06:19<02:24, 2052.60it/s]


0.0 distortion:  76%|█████████████████████████████████████████▋             | 883836/1166243 [06:25<01:57, 2402.00it/s]


0.0 distortion:  77%|██████████████████████████████████████████▍            | 899385/1166243 [06:32<02:02, 2182.44it/s]


0.0 distortion:  78%|██████████████████████████████████████████▋            | 903846/1166243 [06:35<02:13, 1960.61it/s]

In [ ]:
x_distorted

Train MLP

In [ ]:
MLP_path = osp.join(chkpt_path, "MLP.pt")

In [ ]:
if not osp.exists(MLP_path):
    MLP_model.reset_parameters()
    MLP_optimizer = Adam(MLP_model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        loss = train(MLP_model, MLP_optimizer, F.nll_loss, x, y_true, idx["train"])
        acc = test(MLP_model, x, y_true, idx, evaluator)
        print(f"Epoch: {epoch:02d}, "
              f"Loss: {loss:.4f}, "
              f"Train: {100 * acc['train']:.2f}%, "
              f"Valid: {100 * acc['val']:.2f}%, "
              f"Test: {100 * acc['test']:.2f}%")
    torch.save(MLP_model.state_dict(), MLP_path)

Load MLP model

In [ ]:
MLP_model.load_state_dict(torch.load(MLP_path, map_location=device))
z, _, _ = MLP_model(x)

# Verify that the accuracies from all the representations are the same

Input layer

In [ ]:
x_acc = test(MLP_model, x, y_true, idx, evaluator)
print(f"Train: {100 * x_acc['train']:.2f}%, "
      f"Valid: {100 * x_acc['val']:.2f}%, "
      f"Test: {100 * x_acc['test']:.2f}%")

In [ ]:
if do_tsne:
    x_train_fig = plot_tsne(x, idx["train"], 3, y_true[idx["train"]])
    x_train_fig.show()

In [ ]:
if do_tsne:
    x_val_fig = plot_tsne(x, idx["val"], 3, y_true["val"])
    x_val_fig.show()

In [ ]:
if do_tsne:
    x_test_fig = plot_tsne(x, idx["test"], 3, y_true["test"])
    x_test_fig.show()

First hidden layer

In [ ]:
z0_acc = test(MLP_model, x, y_true, idx, evaluator, z[0], layer=1)
print(f"Train: {100 * z0_acc['train']:.2f}%, "
      f"Valid: {100 * z0_acc['val']:.2f}%, "
      f"Test: {100 * z0_acc['test']:.2f}%")

In [ ]:
if do_tsne:
    z0_train_fig = plot_tsne(z[0], idx["train"], 3, y_true["train"])
    z0_train_fig.show()

In [ ]:
if do_tsne:
    z0_val_fig = plot_tsne(z[0], idx["val"], 3, y_true["val"])
    z0_val_fig.show()

In [ ]:
if do_tsne:
    z0_test_fig = plot_tsne(z[0], idx["test"], 3, y_true["test"])
    z0_test_fig.show()

Second hidden layer

In [ ]:
z1_acc = test(MLP_model, x, y_true, idx, evaluator, z[1], layer=2)
print(f"Train: {100 * z1_acc['train']:.2f}%, "
      f"Valid: {100 * z1_acc['val']:.2f}%, "
      f"Test: {100 * z1_acc['test']:.2f}%")

In [ ]:
if do_tsne:
    z1_train_fig = plot_tsne(z[1], idx["train"], 3, y_true["train"])
    z1_train_fig.show()

In [ ]:
if do_tsne:
    z1_val_fig = plot_tsne(z[1], idx["val"], 3, y_true["val"])
    z1_val_fig.show()

In [ ]:
if do_tsne:
    z1_test_fig = plot_tsne(z[1], idx["test"], 3, y_true["test"])
    z1_test_fig.show()

Output layer

In [ ]:
z2_acc = test(MLP_model, x, y_true, idx, evaluator, z[2], layer=3)
print(f"Train: {100 * z2_acc['train']:.2f}%, "
      f"Valid: {100 * z2_acc['val']:.2f}%, "
      f"Test: {100 * z2_acc['test']:.2f}%")

In [ ]:
if do_tsne:
    z2_train_fig = plot_tsne(z[2], idx["train"], 3, y_true["train"])
    z2_train_fig.show()

In [ ]:
if do_tsne:
    z2_val_fig = plot_tsne(z[2], idx["val"], 3, y_true["val"])
    z2_val_fig.show()

In [ ]:
if do_tsne:
    z2_test_fig = plot_tsne(z[2], idx["test"], 3, y_true["test"])
    z2_test_fig.show()

# Reconstruct representations

Noisy node feature matrix

In [ ]:
x_distorted

In [ ]:
for x_type in ["X_c", "X_z"]:
    for perc in range(0, 101, 10):
        reqd_x = x_distorted[x_type][perc].to(device)
        acc = test(MLP_model, reqd_x, y_true, idx, evaluator)
        print(f"{x_type}: n_X: {perc}%, Train: {100 * acc['train']:.4f}%, "
              f"Val: {100 * acc['val']:.4f}%, Test: {100 * acc['test']:.4f}%")
        z, _, _ = MLP_model(reqd_x)
        for split in acc: results[x_type]["phi"][split][perc] = [acc[split], 
                                                                 reqd_x[split], 
                                                                 z[0][split], 
                                                                 z[1][split], 
                                                                 z[2][split]
                                                                ]

Reconstructions of noisy node feature matrix

In [ ]:
x_rbm = B_RBM("Gaussian", x.size(1), x_RBM_num_hid).to(device)
psi_0_res = denoiseRBM(MLP_model, x_rbm, x, x_distorted, idx, device, chkpt_path, x_RBM_num_hid, 
                       x_RBM_num_epochs, x_RBM_lr, x_RBM_CD, x_RBM_batch_size, 10)
for x_type in ["X_c", "X_z"]:
    results[x_type]["psi_0"] = psi_0_res[x_type]

Reconstructions of first hidden layer representations

In [ ]:
z0_rbm = B_RBM("Gaussian", z[0].size(1), z0_RBM_num_hid).to(device)
psi_1_res = denoiseRBM(MLP_model, z0_rbm, z[0], x_distorted, idx, device, chkpt_path, 
                       x_RBM_num_hid, x_RBM_num_epochs, x_RBM_lr, x_RBM_CD, x_RBM_batch_size, 
                       10, layer=1)
for x_type in ["X_c", "X_z"]:
    results[x_type]["psi_1"] = psi_1_res[x_type]

Reconstructions of second hidden layer representations

In [ ]:
z1_rbm = B_RBM("Gaussian", z[1].size(1), z1_RBM_num_hid).to(device)
psi_2_res = denoiseRBM(MLP_model, z1_rbm, z[1], x_distorted, idx, device, chkpt_path, 
                       x_RBM_num_hid, x_RBM_num_epochs, x_RBM_lr, x_RBM_CD, x_RBM_batch_size, 
                       10, layer=2)
for x_type in ["X_c", "X_z"]:
    results[x_type]["psi_2"] = psi_2_res[x_type]

Reconstructions of third hidden layer representations

In [ ]:
z2_rbm = B_RBM("Gaussian", z[2].size(1), z2_RBM_num_hid).to(device)
psi_3_res = denoiseRBM(MLP_model, z2_rbm, z[2], x_distorted, idx, device, chkpt_path, 
                       x_RBM_num_hid, x_RBM_num_epochs, x_RBM_lr, x_RBM_CD, x_RBM_batch_size, 
                       10, layer=3)
for x_type in ["X_c", "X_z"]:
    results[x_type]["psi_3"] = psi_3_res[x_type]

In [ ]:
for x_type in ["X_c", "X_z"]:
    for split in ["train", "val", "test"]:
        for perc in range(0, 101, 10):
            np.savez(osp.join(chkpt, f"{x_type}_{perc}_A_c_0_{split}.npz"),
                     phi_acc = results[x_type]["phi"][split][perc][0],
                     phi_z0 = results[x_type]["phi"][split][perc][1],
                     phi_z1 = results[x_type]["phi"][split][perc][2],
                     phi_z2 = results[x_type]["phi"][split][perc][3],
                     phi_z3 = results[x_type]["phi"][split][perc][4],

                     psi_0_acc = results[x_type]["psi_0"][split][perc][0],
                     psi_0_z0 = results[x_type]["psi_0"][split][perc][1],
                     psi_0_z1 = results[x_type]["psi_0"][split][perc][2],
                     psi_0_z2 = results[x_type]["psi_0"][split][perc][3],
                     psi_0_z3 = results[x_type]["psi_0"][split][perc][4],

                     psi_1_acc = results[x_type]["psi_1"][split][perc][0],
                     psi_1_z1 = results[x_type]["psi_1"][split][perc][1],
                     psi_1_z2 = results[x_type]["psi_1"][split][perc][2],
                     psi_1_z3 = results[x_type]["psi_1"][split][perc][3],

                     psi_2_acc = results[x_type]["psi_2"][split][perc][0],
                     psi_2_z2 = results[x_type]["psi_2"][split][perc][1],
                     psi_2_z3 = results[x_type]["psi_2"][split][perc][2],

                     psi_3_acc = results[x_type]["psi_3"][split][perc][0],
                     psi_3_z3 = results[x_type]["psi_3"][split][perc][1]
                    )